# Number theory neat examples

<span style="font-size: 16pt; font-style: italic; font-weight: bold">Set 2 : </span>
<span style="font-size: 16pt; font-style: italic;">modular exponentiation, divisors, totient function</span>

Anton Antonov    
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com)   
January 2025     

In [ ]:
#%js
    my $m = (12..32).pick;
    my $s = 60;
    my @a = ((2..$s) X (2 .. 2 * $s)).map({ power-mod(|$_, $m) }).rotor(2 * $s-1);
    js-d3-matrix-plot(@a, width => 600, height => 300, :!grid-lines, color-palette => 'Viridis', :!tooltip)

----

## Introduction

**What is a neat example?** : Concise or straightforward code that produces compelling visual or textual outputs.

**Maybe:** We know *neat* when we see it?

The neat examples:

- Showcase Raku programming.
- Use functionalities of different Raku modules.
- Give interesting perspectives on what is computationally possible.

Showcased:
- All number theory functions are provided by ["Math::NumberTheory"](https://raku.land/zef:antononcube/Math::NumberTheory).   
- Visualization functions are provided by ["JavaScript::D3"](https://raku.land/zef:antononcube/JavaScript::D3).
- Data manipulation functions are provided by ["Data::Reshapers"](https://raku.land/zef:antononcube/Data::Reshapers).
- Data summarization functions are provided by ["Data::Summarizers"](https://raku.land/zef:antononcube/Data::Summarizers).
- Data translation functions (like `to-html`) are provided by ["Data::Translators"](https://raku.land/zef:antononcube/Data::Translators).

-----

## Setup

In [2]:
use Math::NumberTheory;
use Math::NumberTheory::Utilities;

In [ ]:
%% javascript
require.config({
     paths: {
     d3: 'https://d3js.org/d3.v7.min'
}});

require(['d3'], function(d3) {
     console.log(d3);
});

In [ ]:
#%js
js-d3-list-line-plot(rand xx 40, background => 'none')

In [ ]:
multi sub highlight-html-table(Str:D $s, @highlight, Str:D :$color = 'Orange', :$font-size = Whatever, :$font-weight = 'normal') { 
    return highlight-html-table($s, :@highlight, :$color, :$font-size);
} 

multi sub highlight-html-table(Str:D $s, :h(:@highlight)!, Str:D :c(:$color) = 'Orange', :s(:$font-size) = Whatever, :w(:$font-weight) = 'normal') { 
    my $head = $font-size ~~ Numeric:D ?? "<span style=\"color: $color; font-size:{$font-size}pt; font-weight:$font-weight\">" !! "<span style=\"color: $color; font-weight:$font-weight\">";
    reduce( 
        { $^a.subst( / <?after '<td>'> $^b <?before '</td>'> /, $head ~ $^b ~ '</span>', :g) }, 
        $s, |@highlight) 
}

----

## Modular exponentiation (power mod)

The function `power-mod(a, b, m)` gives $a^b \mod m$.

First, there is a function `prime(n)` that gives the n-th prime number:

In [ ]:
[10, 20 ... 100]».&prime

Plot a list of powers of 3 where the exponent is varied, modulo some prime number:

In [ ]:
#%js
my @a = (1 ... (prime(44)-1) ).map({ power-mod(3, $_ , prime(44)) });
js-d3-list-plot(@a, 
    title-color => 'DimGray',
    plot-label => "3ᵇ mod {prime(44)}",
    background => 'none',
    width => 800,
    height => 200
)

Plot values of varying powers of numbers with a fixed modulus:

In [ ]:
#%js
my @a = ((2..100) X (2..100)).map({ power-mod($_.head, $_.tail, 32) }).rotor(99);
js-d3-matrix-plot(@a, width => 400, height => 400, :!grid-lines)

Plot an a version of [Ulam spiral](https://en.wikipedia.org/wiki/Ulam_spiral) where numbers are colored based on `power-mod`:

In [ ]:
#% js
my @mat = spiral-lattice(101).deepmap({ power-mod($_, 3, 17) });
js-d3-matrix-plot(@mat, width => 400, height => 400, :!grid-lines, :!tooltip, color-palette => 'Rainbow')

----

## Modular inverse

The function `modular-inverse(k,n)` gives the modular inverse of $k$ modulo $n$.

- `modular-inverse` is also known as modular multiplicative inverse.
- Typically used in modular arithmetic and cryptography.
- `modular-inverse(k,n)` gives the number $r$ such that the remainder of the division of $r k$ by $n$ is equal to $1$.
- If $k$ and $n$ are not coprime, no modular inverse exists and `modular-inverse(k,n)` returns `Nil`.

In [ ]:
#%html
my @row = 2, 3, 5, 7;
my @col = 1..10;
(@row X @col).map({ $_.tail => modular-inverse($_.tail, $_.head) // '' }).rotor(@col.elems)».Hash
==> { $_.kv.map(-> $k, %v { %v<n> = @row[$k]; %v }) }()
==> to-html(field-names => ['n', |@col».Str])

Visualize when a number is invertible modulo 12:

In [ ]:
#% js
my $n = 10;
my $m = 12;
my @a = ((1..$n) X (1..$n)).map({ not so modular-inverse($_.head + $_.tail ** 2, $m) })».Int.rotor($n);

js-d3-matrix-plot(@a, width => 400, height => 400, :!grid-lines, color-palette => 'Blues', :!tooltip)

------

## Divisor sigma

The function `divisor-sigma(k,n)` gives the divisor function $\sigma_{k}(n)$.

- `divisor-sigma` is also known as the divisor function or sum‐of‐divisors function.
- `divisor-sigma(k,n)` is the sum of the $k^{th}$ powers of the divisors of $n$.
- For a number $n = u \times p_1^{e_1} ... p_{m}^{e_{m}}$ with a unit $u$ and $p_{i}$ primes, `divisor-sigma(k, n)` returns $(1 + p_1^{k} + p_1^{2 k} + ... + p_1^{e_1 k} )(1 + p_2^{k} + p_2^{2 k} + ... + p_1^{e_2 k})...$.

In [ ]:
#%html
my @cols = 1..3;
(@cols X (^11)).map({ $_.tail => divisor-sigma($_.tail, $_.head) }).rotor(11)».Hash
==> { $_.kv.map(-> $k, %v { %v<n> = @cols[$k]; %v }) }()
==> to-html(field-names => ['n', |(^11)».Str])

In [ ]:
my @cols = "σ(0) Number of divisors", "σ(1) Sum of divisors", "σ(2) Sum of squares of divisors";
my @data = ((^3) X (1..50)).map({ <group x y> Z=> [@cols[$_.head], $_.tail, divisor-sigma($_.tail, e => $_.head).log] })».Hash;

deduce-type(@data)

In [ ]:
#%js
js-d3-list-line-plot(@data,
    background => 'none',
    width => 1000,
    height => 300
)

Plot a version of [Ulam spiral](https://en.wikipedia.org/wiki/Ulam_spiral) with the number divisors:

In [ ]:
#% js
my @mat = spiral-lattice(71).deepmap({ divisor-sigma(0, $_)});
js-d3-matrix-plot(@mat, width => 400, height => 400, :!grid-lines, color-palette => 'Plasma', :!tooltip)

-----

## Primitive root

Elements relatively prime to 22 are enumerated by the primitive root:

In [ ]:
my $n = 2 * prime(5);
my $p = primitive-root($n);

(:$n, :$p)

Make the edges of a weighted graph -- the weights show the number of times `power-mod` jumps from one power to another:

In [ ]:
my @edges = (1..$n).map({ power-mod($p, $_, $n).Str => power-mod($p, $_ + 1, $n).Str }).classify(*).map({ %(from => $_.key.key, to => $_.key.value, weight => $_.value.elems) });

deduce-type(@edges)

Plot the graph:

In [ ]:
#%html
my $g = Graph.new(@edges, :directed-edges);

$g.dot(background => 'none', engine => 'neato'):svg

----

## Prime Pi

- `prime-pi` gives the number of primes $\pi(x)$ less than or equal to $x$.
- $\pi(x)$ is also known as prime counting function.
- $\pi(x)$ has the asymptotic expansion $x/log(x) + x/log^2(x)+2 x/log^3(x) + ... $ as $x \rightarrow \propto$.

In [ ]:
#% js
my $limit = 200;
my @data = (2..$limit).map({ %( group => 'x/log(x) + x/log(x)²', x => $_, y => $_ / log($_) +  $_ / log($_) ** 2 ) });
@data .= append: (2..$limit).map({ %( group => 'π(x)', x => $_, y => prime-pi($_) ) });
js-d3-list-line-plot(@data, background => 'none', width => 800, height => 300)

[Ulam spiral](https://en.wikipedia.org/wiki/Ulam_spiral) colored based on the difference in `prime-pi` values: 

In [ ]:
my $method = 'Legendre';
my @mat = spiral-lattice(101).deepmap({ prime-pi($_ + 10, :$method) - prime-pi($_, :$method) });

deduce-type(@mat)

Plot the spiral:

In [ ]:
#% js
js-d3-matrix-plot(@mat, width => 400, height => 400, :!grid-lines, color-palette => 'Rainbow', :!tooltip)

----

## Totient function 

The totient function $\phi(x)$, also called Euler's totient function, is defined as the number of positive integers $≤ n$ that are relatively prime to (i.e., do not contain any factor in common with) $n$, where $1$ is counted as being relatively prime to all numbers. 

- `euler-phi` is also known as the [Euler totient function](https://mathworld.wolfram.com/TotientFunction.html) or _phi function_.
- Integer mathematical function suitable for numerical manipulation.
- Typically used in cryptography and in many applications in elementary number theory.
- `euler-phi(n)` counts positive integers up to `n` that are relatively prime to `n`.
- For the number $n = u \times p_1^{k_1} ... p_{m}^{m_1}$ with $u$ unit and $p_{i}$ primes, `euler-phi(n)` gives $n (1 - \frac{1}{p_1}) ... (1 - \frac{1}{p_{m}})$.

In [ ]:
#% js
js-d3-list-line-plot((^100)».&euler-phi, background => 'none', plot-label => "Euler's ϕ(x)", title-color => 'DimGray')

$\phi(n)$ satisfies:

$$
\liminf_{n \rightarrow \propto} \phi(n) \frac{\ln \ln n}{n} = e^{-\gamma} 
$$

See [MathWorld's "Totient Function"](https://mathworld.wolfram.com/TotientFunction.html).

In [ ]:
#% js
js-d3-list-plot(
    (2..5000).map({ euler-phi($_) * $_.log.log / $_ }), 
    background => 'none', 
    title => "Euler's ϕ(x)", 
    title-color => 'DimGray',
    point-size => 2
)

Plot of Ulam spiral where numbers are colored based on the values of `euler-phi`:

In [ ]:
my $n = 141;
my @mat = spiral-lattice($n).&flatten.hyper(:4degree).map({ euler-phi($_) }).rotor($n);

@mat ==> dimensions

In [ ]:
#% js
js-d3-matrix-plot(@mat, width => 400, height => 400, :!grid-lines, color-palette => 'Viridis', :!mesh, :!tooltip)